In [166]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import datetime
import random
import matplotlib.dates as mdates

def MakeDataFromText(filepath):
    """
    Helper function to parse text files in IFM proprietary format.
    """
    with open(filepath, 'r') as file:
        reader = csv.reader(file)

        db = dict()                          # Dictionary that will hold k,v pairs of 
        recent_EPC = ''                      # The most recent EPC that was read, and data is being assigned to
        Time_Or_Data = True                  # Time is true, Data is false
        reached_24 = False

        for l in reader:                     # Iterate over entire text file
            line = l[0]                      # Each line only has one token, so we select it
            
            if len(line)==24:                # If it is an EPC
                db[line] = [[],[]]           # Create new k,v pair in database
                recent_EPC = line            # Current EPC to add to is selected
            
            elif line == '[':                # If start bracket, beginning to collect an array of data
                data = []

            elif line == ']':                # End bracket means end of data array
                if len(data)!=0:
                    if Time_Or_Data:
                        db[recent_EPC][0] = data
                        Time_Or_Data = False
                    else:
                        db[recent_EPC][1] = data
                        Time_Or_Data = True
                data = []

            elif len(line)==8:               # If it is a time stamp
                h,m,s = line.split(':')      # Parse timestamp into hours, minutes, seconds

                if int(h)==23:
                    reached_24 = True        # If end of day reached, must move to Day 2

                if reached_24 and int(h) < 12:
                    day=2
                else:
                    day=1

                data.append(datetime.datetime(1900, 1, day,int(h) - 3,int(m),int(s))) # Minus 3 for Pacific Time
            
            elif line ==' ':               # If blank space, then we skip to parse next line
                pass

            else:                          # If it is a data point
                try:
                    data.append(float(line))
                except:
                    pass                   # Error catching case just in case, so annoying errors don't crash program
    return db

class Person:
    """
    Class to represent each individal poutfit being recorded during Deployment, made up of garments.
    """
    def __init__(
        self, wet:bool, Name:str, color:str,
        backneck:str,   back:str,        chest:str, 
        leftab:str,     rightab:str, 
        rightuparm:str, rightlowarm:str, 
        leftuparm:str,  leftlowarm:str, 
        beanie1:str,    beanie2:str,     beanie3:str, beanieR:str, beanieL:str
    ):
        self.Backneck    = backneck        # Tag ID's for the shirt
        self.Back        = back
        self.Chest       = chest
        self.Leftab      = leftab
        self.Rightab     = rightab
        self.Rightuparm  = rightuparm
        self.Rightlowarm = rightlowarm
        self.Leftuparm   = leftuparm
        self.Leftlowarm  = leftlowarm
        self.Beanie1     = beanie1         # Tag ID's for the beanies
        self.Beanie2     = beanie2
        self.Beanie3     = beanie3
        self.BeanieR     = beanieR
        self.BeanieL     = beanieL

        self.Name = Name                   # Soldier's Name

        self.ShirtMapping = {
            self.Backneck:    Name + " Back Neck",
            self.Back:        Name + " Back",
            self.Chest:       Name + " Chest",
            self.Leftab:      Name + " Left Abdomen",
            self.Rightab:     Name + " Right Abdomen",
            self.Rightuparm:  Name + " Right Upper Arm",
            self.Rightlowarm: Name + " Right Lower Arm",
            self.Leftlowarm:  Name + " Left Lower Arm",
            self.Leftuparm:   Name + " Left Upper Arm",
        }

        self.TagList = [
            self.Backneck,   self.Back, self.Chest,
            self.Leftab,     self.Rightab,
            self.Rightuparm, self.Rightlowarm,
            self.Leftuparm,  self.Leftlowarm,
            self.Beanie1,    self.Beanie2, self.Beanie3, self.BeanieR, self.BeanieL
        ]

        self.BeanieList = [self.Beanie1, self.Beanie2, self.Beanie3, self.BeanieR, self.BeanieL]
        self.Color = color

        self.wet = wet # Bool for wetness check
        if wet:
            self.BackColor = "#870101"
            self.BackNeckColor = "#ad4503"
            self.ChestColor = "#756a01"
            self.LeftAbColor = "#396102"
            self.RightAbColor = "#015c0b"
            self.LeftUpArmColor = "#01663f"
            self.RightUpArmColor = "#00a191"
            self.LeftLowArmColor = "#012173"
            self.RightLowArmColor = "#1e075e"
            
        else:
            self.BackColor = "#ff0d0d"
            self.BackNeckColor = "#ff7214"
            self.ChestColor = "#d1bc02"
            self.LeftAbColor = "#6bb803"
            self.RightAbColor = "#01bf16"
            self.LeftUpArmColor = "#03c278"
            self.RightUpArmColor = "#06b4cf"
            self.LeftLowArmColor = "#0443e0"
            self.RightLowArmColor = "#4c05e6"

In [167]:
# Paul P, only wet
person6  = Person(True, "Paul P.", "#de0413", "71BB", "7705", "B25E", "3247", "A9B5", "6C38", "7662", "A983", "098F", "B644", "7BCC", "B576", "70D1", "4D84");

# Joelle, only wet
person14 = Person(True, "Joelle", "#9e05a1", "462C", "5B60", "7415", "6310", "1851", "616A", "5DAA", "6D28", "9991", "3B0E", "5374", "682C", "0A50", "1543");

# Garrett C, only wet
person28 = Person(True, "Garrett C.", "#0909eb", "8977", "4EB3", "AA5E", "6626", "9ABE", "6CD0", "AB8E", "14AA", "6E94", "895D", "571F", "2D68", "8121", "198C");

# Dennis, only wet
person2  = Person(True, "Dennis", "#078c26", "333B", "289B", "2473", "231D", "9879", "4067", "5FB6", "169E", "8D50", "74C4", "73DC", "DC4A", "884D", "1BA4");

# Brent, only wet
person17 = Person(True, "Brent L.", "#e8c100", "6CC7", "8FB7", "799D", "6F8D", "6332", "5A1E", "92A5", "4A3D", "3EC9", "16AB", "A66C", "6162", "A686", "7E1B");

In [168]:
t1 = MakeDataFromText("MWTC/trevor_test_1.txt")
t2 = MakeDataFromText("MWTC/trevor_test_2.txt")
t3 = MakeDataFromText("MWTC/trevor_test_3.txt")
t4 = MakeDataFromText("MWTC/trevor_test_4.txt")
t5 = MakeDataFromText("MWTC/trevor_test_5.txt")
t6 = MakeDataFromText("MWTC/trevor_test_6.txt")
t7 = MakeDataFromText("MWTC/trevor_test_7.txt")
t8 = MakeDataFromText("MWTC/trevor_test_8.txt")
y1 = MakeDataFromText("MWTC/yoel_test_1.txt")
y2 = MakeDataFromText("MWTC/yoel_test_2.txt")
y3 = MakeDataFromText("MWTC/yoel_test_3.txt")
y4 = MakeDataFromText("MWTC/yoel_test_4.txt")

combined = dict()

for d in [t1, t2, t3, t4, t5, t6, t7, t8, y1, y2, y3, y4]:

    for k,v in d.items():
        if k in combined: # if tag is already in dict
            
            for i in range(len(v[0])):
                if v[0][i] in combined[k]:
                    combined[k][v[0][i]].append(v[1][i])
                else:
                    combined[k][v[0][i]] = [v[1][i]]
        
        else: # if tag is not in dict
            combined[k] = dict()
            for i in range(len(v[0])):
                combined[k][v[0][i]] = [v[1][i]]

for k,v in combined.items():
    for k2,v2 in v.items():
        combined[k][k2] = sum(v2)/len(v2)

In [169]:
Back_sum = 0
neck_sum = 0
larm_sum= 0
rarm_sum= 0
low_sum = 0
row_sum = 0
lab_sum = 0
rab_sum = 0
chst_sum = 0
b1_sum = 0
b2_sum = 0
b3_sum = 0
br_sum = 0
bl_sum = 0

p = person28
for d in [t1, t2, t3, t4, t5, t6, t7, t8, y1, y2, y3, y4]:
    for k,v in d.items():
        id = k[-4:]

        for i in range(len(v[0])):
            if v[0][i]>datetime.datetime(1900, 1, 1, 8, 20, 0) and v[0][i]<datetime.datetime(1900, 1, 1, 8, 41, 0):

                if id==p.Back:
                    Back_sum += 1
                
                elif id==p.BeanieR:
                    br_sum +=1
                
                elif id==p.Leftuparm:
                    larm_sum +=1
                
                elif id==p.Beanie3:
                    b3_sum +=1
                
                elif id==p.Backneck:
                    neck_sum +=1
                
                elif id==p.Rightuparm:
                    rarm_sum +=1
                
                elif id==p.Chest:
                    chst_sum +=1
                
                elif id==p.Leftab:
                    lab_sum +=1
                
                elif id==p.Rightab:
                    rab_sum +=1
                
                elif id==p.Rightlowarm:
                    row_sum +=1
                    
                elif id==p.Beanie2:
                    b2_sum +=1
                    
                elif id==p.Leftlowarm:
                    low_sum +=1
                    
                elif id==p.Beanie1:
                    b1_sum +=1
                
                elif id==p.BeanieL:
                    bl_sum +=1

print("Back: ", Back_sum)
print("Backneck: ", neck_sum)
print("Leftuparm: ", larm_sum)
print("leftlowarm", low_sum)
print("Rightuparm: ", rarm_sum)
print("Rightlowarm: ", row_sum)
print("Leftabdomen: ", lab_sum)
print("Rightabdomen: ", rab_sum)
print("Chest", chst_sum)
print("beanie1", b1_sum)
print("beanie2", b2_sum)
print("beanie3", b3_sum)
print("beanieL", bl_sum)
print("beanieR", br_sum)

Back:  41
Backneck:  49
Leftuparm:  110
leftlowarm 17
Rightuparm:  71
Rightlowarm:  31
Leftabdomen:  16
Rightabdomen:  37
Chest 65
beanie1 24
beanie2 7
beanie3 12
beanieL 9
beanieR 0


In [170]:
Back_sum1 = []
Back_sum2 = []
neck_sum1 = []
neck_sum2 = []
larm_sum1 = []
larm_sum2 = []
rarm_sum1 = []
rarm_sum2 = []
low_sum1, low_sum2, row_sum1, row_sum2 = [],[],[],[]
lab_sum1 = []
lab_sum2 = []
rab_sum1 = []
chst_sum1 = []
rab_sum2 = []
chst_sum2 = []
b1_sum1, b1_sum2 = [],[]
b2_sum1, b2_sum2 = [],[]
b3_sum1, b3_sum2 = [],[]
bR_sum1, bR_sum2 = [],[]
bL_sum1, bL_sum2 = [],[]


p = person28
for k,v in combined.items():
    id = k[-4:]
    if id in p.TagList:
        s = sorted( v.items() )

        pre  = [i[1] for i in s if i[0]>datetime.datetime(1900, 1, 1, 8, 0, 0) and i[0]<datetime.datetime(1900, 1, 1, 8, 19, 30)]
        post = [i[1] for i in s if i[0]>datetime.datetime(1900, 1, 1, 8, 20, 0) and i[0]<datetime.datetime(1900, 1, 1, 8, 41, 0)]

        # print(pre)
        # print(post)

        if id==p.Back:
            for j in pre:
                Back_sum1.append(j)
            for j in post:
                Back_sum2.append(j)
        
        elif id==p.BeanieR:
            for j in pre:
                bR_sum1.append(j)
            for j in post:
                bR_sum2.append(j)
        
        elif id==p.Leftuparm:
            for j in pre:
                larm_sum1.append(j)
            for j in post:
                larm_sum2.append(j)
        
        elif id==p.Beanie3:
            for j in pre:
                b3_sum1.append(j)
            for j in post:
                b3_sum2.append(j)
        
        elif id==p.Backneck:
            for j in pre:
                neck_sum1.append(j)
            for j in post:
                neck_sum2.append(j)
        
        elif id==p.Rightuparm:
            for j in pre:
                rarm_sum1.append(j)
            for j in post:
                rarm_sum2.append(j)
        
        elif id==p.Chest:
            for j in pre:
                chst_sum1.append(j)
            for j in post:
                chst_sum2.append(j)
        
        elif id==p.Leftab:
            for j in pre:
                lab_sum1.append(j)
            for j in post:
                lab_sum2.append(j)
        
        elif id==p.Rightab:
            for j in pre:
                rab_sum1.append(j)
            for j in post:
                rab_sum2.append(j)
        
        elif id==p.Rightlowarm:
            for j in pre:
                row_sum1.append(j)
            for j in post:
                row_sum2.append(j)
        
        elif id==p.Beanie2:
            for j in pre:
                b2_sum1.append(j)
            for j in post:
                b2_sum2.append(j)
        
        elif id==p.Leftlowarm:
            for j in pre:
                low_sum1.append(j)
            for j in post:
                low_sum2.append(j)
        
        elif id==p.Beanie1:
            for j in pre:
                b1_sum1.append(j)
            for j in post:
                b1_sum2.append(j)
        
        elif id==p.BeanieL:
            for j in pre:
                bL_sum1.append(j)
            for j in post:
                bL_sum2.append(j)

In [171]:
print("Back", round(sum(Back_sum1)/len(Back_sum1),2), round(sum(Back_sum2)/len(Back_sum2),2))
print("neck", round(sum(neck_sum1)/len(neck_sum1),2), round(sum(neck_sum2)/len(neck_sum2),2))

print("Left up arm", round(sum(larm_sum1)/len(larm_sum1),2), round(sum(larm_sum2)/len(larm_sum2),2))
print("Right up arm", round(sum(rarm_sum1)/len(rarm_sum1),2), round(sum(rarm_sum2)/len(rarm_sum2),2))

print("Left low arm", round(sum(low_sum1)/len(low_sum1),2), round(sum(low_sum2)/len(low_sum2),2))
print("Right low arm", round(sum(row_sum1)/len(row_sum1),2), round(sum(row_sum2)/len(row_sum2),2))

Back 16.96 9.37
neck 19.75 11.15
Left up arm 22.1 7.91
Right up arm 18.65 5.82
Left low arm 31.79 11.43
Right low arm 30.46 8.94


In [172]:
print("Left ab", round(sum(lab_sum1)/len(lab_sum1),2), round(sum(lab_sum2)/len(lab_sum2),2))
print("Right ab", round(sum(rab_sum1)/len(rab_sum1),2), round(sum(rab_sum2)/len(rab_sum2),2))

Left ab 12.37 6.26
Right ab 11.8 3.1


In [173]:
print("chest", round(sum(chst_sum1)/len(chst_sum1),2), round(sum(chst_sum2)/len(chst_sum2),2))

chest 15.55 6.83


In [174]:
print("beanie 1", round(sum(b1_sum1)/len(b1_sum1),2), round(sum(b1_sum2)/len(b1_sum2),2))
print("beanie 2", round(sum(b2_sum1)/len(b2_sum1),2), round(sum(b2_sum2)/len(b2_sum2),2))
print("beanie 3", round(sum(b3_sum1)/len(b3_sum1),2), round(sum(b3_sum2)/len(b3_sum2),2))


beanie 1 23.04 8.69
beanie 2 22.51 13.24
beanie 3 25.03 12.12


In [176]:
print("beanie L", round(sum(bL_sum1)/len(bL_sum1),2), round(sum(bL_sum2)/len(bL_sum2),2))
print("beanie R", round(sum(bR_sum1)/len(bR_sum1),2), round(sum(bR_sum2)/len(bR_sum2),2))

beanie L 27.38 12.01
beanie R 26.82
